# Launch dask cluster

We first launch a cluster and import it in the notebook, it should look like this :

# Import libraries

We load basic libraries (numpy, xarray, matplotlib) and one more specific :
  - gcm_filters for spatial filtering compatible with xarray : https://gcm-filters.readthedocs.io/en/latest/


In [ ]:
import numpy as np
import xarray as xr
import gcm_filters
import matplotlib.colors as clr
import matplotlib.pyplot as plt
%matplotlib inline

# Call data

We use the catalog to import our dataset

In [ ]:
from validate_catalog import all_params
params_dict, cat = all_params()
params_dict.keys()

In [ ]:
enatl01w = cat.eNATL60(region='1',datatype='surface_hourly', season='fma'
                      ).to_dask()

enatl01w

# Set up variables needed for ${\tt gcm-filters}$

In [ ]:
wet_mask = xr.DataArray(np.zeros_like(enatl01w.sosstsst.isel(time_counter=0)),
                        dims=['y','x'],
                        coords={'y':np.arange(len(enatl01w.y)),
                                'x':np.arange(len(enatl01w.x))}
           )
wet_mask[1:-1,1:-1] = enatl01w.tmask[1:-1,1:-1].data.astype(np.float64)

wet_mask.plot()

In [ ]:
sst = xr.DataArray(.5*(enatl01w.sosstsst.fillna(0.).data 
                       + enatl01w.sosstsst.fillna(0.).data
                      ),
                   dims=['time_counter','y','x'],
                   coords={'time_counter':enatl01w.time_counter.data, 
                           'y':np.arange(len(enatl01w.y)),
                           'x':np.arange(len(enatl01w.x))}
                  )

In [ ]:
area = (enatl01w.e1t * enatl01w.e2t)

area.plot()

In [ ]:
dxw = xr.DataArray(enatl01w.e1u.data, dims=['y','x'],
                   coords={'y':np.arange(len(enatl01w.y)),
                           'x':np.arange(len(enatl01w.x))}
                  ) # x-spacing centered at western cell edge

dyw = xr.DataArray(enatl01w.e2u.data, dims=['y','x'],
                   coords={'y':np.arange(len(enatl01w.y)),
                           'x':np.arange(len(enatl01w.x))}
                  ) # y-spacing centered at western cell edge

dxs = xr.DataArray(enatl01w.e1v.data, dims=['y','x'],
                   coords={'y':np.arange(len(enatl01w.y)),
                           'x':np.arange(len(enatl01w.x))}
                  ) # x-spacing centered at southern cell edge

dys = xr.DataArray(enatl01w.e2v.data, dims=['y','x'],
                   coords={'y':np.arange(len(enatl01w.y)),
                           'x':np.arange(len(enatl01w.x))}
                  ) # y-spacing centered at southern cell edge
dxw

In [ ]:
dx_min = min(enatl01w.e1u.min(['y','x']), enatl01w.e2u.min(['y','x']),
             enatl01w.e1v.min(['y','x']), enatl01w.e2v.min(['y','x'])
            )
dx_min = dx_min.values
dx_min

In [ ]:
kappa_w = xr.zeros_like(wet_mask)
kappa_w[1:-1,1:-1] += 1.
kappa_s = xr.zeros_like(wet_mask)
kappa_s[1:-1,1:-1] += 1.

In [ ]:
filter_scale = 30000   # 30 km

filter_30km = gcm_filters.Filter(
    filter_scale=filter_scale,
    dx_min=dx_min,
    # n_steps=36,
    filter_shape=gcm_filters.FilterShape.GAUSSIAN,
    grid_type=gcm_filters.GridType.IRREGULAR_WITH_LAND,
    grid_vars={
        'wet_mask': wet_mask.chunk({'y':-1,'x':-1}), 
        'dxw': dxw.chunk({'y':-1,'x':-1}), 'dyw': dyw.chunk({'y':-1,'x':-1}), 
        'dxs': dxs.chunk({'y':-1,'x':-1}), 'dys': dys.chunk({'y':-1,'x':-1}), 
        'area': area.chunk({'y':-1,'x':-1}), 
        'kappa_w': kappa_w.chunk({'y':-1,'x':-1}), 'kappa_s': kappa_s.chunk({'y':-1,'x':-1})
    }
)

filter_30km

# Filter SST

In [ ]:
sst_filtered_to_30km = filter_30km.apply(sst.chunk({'y':-1,'x':-1}), dims=['y', 'x'])

fig, ax = plt.subplots()
(sst_filtered_to_30km.where(enatl01w.tmask!=0.).isel(time_counter=0)
).plot(vmax=4,vmin=26,cmap='magma',ax=ax)
ax.set_aspect('equal')

In [ ]:
vmin = 4
vmax = 26

fig, axs = plt.subplots(1,3,figsize=(16,6))

im = axs[0].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                       enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)),
                       sst.where(enatl01w.tmask!=0.).isel(time_counter=0, 
                                                          y=slice(1,-1),x=slice(1,-1)), 
                       vmin=vmin, vmax=vmax, 
                       cmap='magma', 
#                        cbar_kwargs={'label': r'm d$^{-1}$'},
                       shading='auto', rasterized=True
)
axs[0].set_title('unfiltered SST', fontsize=15)

axs[1].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                  enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)),
                  sst_filtered_to_30km.where(enatl01w.tmask!=0.).isel(time_counter=0,
                                                                      y=slice(1,-1),x=slice(1,-1)), 
                  vmin=vmin, vmax=vmax, 
                  cmap='magma', 
#                   cbar_kwargs={'label': r'm d$^{-1}$'}
                  shading='auto', rasterized=True
)
axs[1].set_title('SST filtered to 30km', fontsize=15)

im2 = axs[2].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                        enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)),
                        (sst.where(enatl01w.tmask!=0.).isel(time_counter=0) 
                         - sst_filtered_to_30km.where(enatl01w.tmask!=0.).isel(time_counter=0)
                        ).isel(y=slice(1,-1),x=slice(1,-1)), 
                        vmax=1, vmin=-1,
                        cmap='bwr', 
#                   cbar_kwargs={'label': r'm d$^{-1}$'}
                        shading='auto', rasterized=True
)
axs[2].set_title(r'(unfiltered - filtered) SST', fontsize=15);

axs[0].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                  enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)), 
                  np.ma.masked_values(enatl01w.tmask.isel(y=slice(1,-1),x=slice(1,-1)), 
                                      1.).astype(np.float64),
                  vmin=-1, vmax=2, cmap='binary', rasterized=True)
axs[1].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                  enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)), 
                  np.ma.masked_values(enatl01w.tmask.isel(y=slice(1,-1),x=slice(1,-1)), 
                                      1.).astype(np.float64),
                  vmin=-1, vmax=2, cmap='binary', rasterized=True)
axs[2].pcolormesh(enatl01w.lon.isel(y=slice(1,-1),x=slice(1,-1)), 
                  enatl01w.lat.isel(y=slice(1,-1),x=slice(1,-1)), 
                  np.ma.masked_values(enatl01w.tmask.isel(y=slice(1,-1),x=slice(1,-1)), 
                                      1.).astype(np.float64),
                  vmin=-1, vmax=2, cmap='binary', rasterized=True)

axs[0].set_ylabel(r"Latitude", fontsize=15)
axs[0].set_xlabel(r"Longitude", fontsize=15)
axs[1].set_xlabel(r"Longitude", fontsize=15)
axs[2].set_xlabel(r"Longitude", fontsize=15)

axs[0].set_aspect('equal')
axs[1].set_aspect('equal')
axs[2].set_aspect('equal')

fig.subplots_adjust(bottom=0.255, hspace=.2)
cbaxes = fig.add_axes([0.2, 0.15, 0.35, 0.019]) 
cbar = fig.colorbar(im, orientation='horizontal', cax=cbaxes,
#                     ticks=[-2.8,-2.1,-1.4,-.7,0.,.7,1.4,2.1,2.8]
                   )
cbaxes2 = fig.add_axes([0.711, 0.15, 0.15, 0.019]) 
cbar2 = fig.colorbar(im2, orientation='horizontal', cax=cbaxes2,
#                     ticks=[-2.8,-2.1,-1.4,-.7,0.,.7,1.4,2.1,2.8]
                    )
# cbar.set_powerlimits(-1,1)
cbar.set_label(r'[$^\circ$C]', fontsize=14)
cbar2.set_label(r'[$^\circ$C]', fontsize=14)